In [43]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import random

In [45]:
df = pd.read_csv('../Dataset/dataset_1.csv')
df
# df = pd.DataFrame(dataset)
# df['label'] = df['label'].map({-1: "tidak puas", 0: "netral", 1: "puas"})


,Ulasan,Label
0,barang,Neutral
1,standard sepatu harga segitu,Neutral
2,sayang ukur standard internasional standard in...,Neutral
3,miring kak,Neutral
4,ekspektasi,Neutral
...,...,...
715,sepatu belah kiri tekuk utk tumit,Tidak Puas
716,kecewa sih packingnya dibungkus plastik transp...,Tidak Puas
717,jelek barangnya,Tidak Puas
718,noda kayak sepatu bekas,Tidak Puas


In [46]:
# Ubah label -1 menjadi 2
df['Label'] = df['Label'].map({"Puas": 2, "Neutral": 1, "Tidak Puas": 0})

# Validasi Label
assert df['Label'].isin([0, 1, 2]).all(), "Ada Label yang tidak valid"
print(df['Label'].unique())  # Output harus [0, 1, 2]


[1 2 0]


In [47]:
# Tokenisasi Data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

df['tokenized'] = df['Ulasan'].apply(lambda x: tokenize_function(x))


In [48]:
# Split Dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Ulasan'], df['Label'], test_size=0.2, random_state=42)


In [49]:
# Buat Dataset PyTorch
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)

train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
val_dataset = SentimentDataset(val_encodings, val_labels.tolist())


In [50]:
# Ubah BERT dan Trainer ke mode GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [54]:

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to('cuda')
training_args = TrainingArguments(
    output_dir='../results/v_' + str(versi_model),
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=250,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=200,
    save_total_limit=2,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  3%|▎         | 20/720 [00:09<05:51,  1.99it/s]

{'loss': 1.1096, 'grad_norm': 6.120259761810303, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.28}


  6%|▌         | 40/720 [00:20<05:43,  1.98it/s]

{'loss': 1.1123, 'grad_norm': 5.1496686935424805, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.56}


  8%|▊         | 60/720 [00:30<05:44,  1.92it/s]

...


 97%|█████████▋| 701/720 [03:45<00:06,  3.06it/s]

{'eval_loss': 0.9639005661010742, 'eval_accuracy': 0.8402777777777778, 'eval_f1': 0.8371817260135618, 'eval_precision': 0.8360578127195314, 'eval_recall': 0.8400247413405308, 'eval_runtime': 0.4604, 'eval_samples_per_second': 312.745, 'eval_steps_per_second': 10.859, 'epoch': 9.72}


100%|██████████| 720/720 [03:48<00:00,  3.15it/s]

{'loss': 0.0017, 'grad_norm': 0.022031528875231743, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 228.9097, 'train_samples_per_second': 25.163, 'train_steps_per_second': 3.145, 'train_loss': 0.36985638414302635, 'epoch': 10.0}


TrainOutput(global_step=720, training_loss=0.36985638414302635, metrics={'train_runtime': 228.9097, 'train_samples_per_second': 25.163, 'train_steps_per_second': 3.145, 'train_loss': 0.36985638414302635, 'epoch': 10.0})

In [55]:
# Evaluasi model setelah pelatihan
eval_result = trainer.evaluate()
eval_result

100%|██████████| 5/5 [00:00<00:00, 13.99it/s]


{'eval_loss': 0.9658242464065552,
 'eval_accuracy': 0.8402777777777778,
 'eval_f1': 0.8371817260135618,
 'eval_precision': 0.8360578127195314,
 'eval_recall': 0.8400247413405308,
 'eval_runtime': 0.4528,
 'eval_samples_per_second': 318.043,
 'eval_steps_per_second': 11.043,
 'epoch': 10.0}

In [56]:

# Save model
versi_model=5
print("Model berhasil disimpan di direktori ../results/" + "v_" + str(versi_model))
model.save_pretrained("../results/" + "v_" + str(versi_model))
tokenizer.save_pretrained("../results/" + "v_" + str(versi_model))

Model berhasil disimpan di direktori ../results/v_5


('../results/v_5\\tokenizer_config.json',
 '../results/v_5\\special_tokens_map.json',
 '../results/v_5\\vocab.txt',
 '../results/v_5\\added_tokens.json')